In [97]:
import pandas as pd
import numpy as np

In [98]:
import urllib.parse
from bs4 import BeautifulSoup

In [99]:
import requests
import brotli

In [100]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By


In [101]:
filein_path = "../../../data/data_wikis/1 - politician data for test scraping/"

In [102]:
fileout_path = "../../../data/data_wikis/2.1 - sk namuwiki scraping test/"

# import sk & tw politician test data

In [140]:
sktest = pd.read_excel(filein_path+"sk_testset.xlsx")

In [141]:
sktest.shape

(10114, 24)

In [142]:
# sktest["URL_NamuWiki"]=np.nan

In [143]:
sktest

,Name_Native,Name_Chinese,Birthdate,Sex,Age,ElectionCategory_Native,ElectionDate,Position,L1_Division,L2_Division,...,CandidateStatus,WonElection,VoteCount,VotePercent,IncumbentStatus,Occupation,Education,Career1,Career2,URL_NamuWiki
0,가세로,賈世魯,19550815,남,58,구·시·군의 장선거,20140604,Chief,충청남도,태안군,...,등록,0,NaN,NaN,NaN,정당인,단국대학교 대학원 행정학박사,(전)박근혜대통령후보 대선 서산.태안총괄본부장,"(현)""박근혜를 사랑하는 모임"" 전국 상임고문",https://namu.wiki/w/%EA%B0%80%EC%84%B8%EB%A1%9C
1,가창현,賈昌鉉,19550315,남,59,구·시·군의회의원선거,20140604,Legislator,충청남도,태안군,...,등록,0,NaN,NaN,NaN,정치인,서산농림고등학교 졸업(현 서산중앙고등학교),(전)태안군환경산림과장,(전)태안군사회복지과장,False
2,감정선,甘貞善,19650919,여,48,시·도의회의원선거,20140604,Legislator,경상남도,창원시의창구,...,등록,0,NaN,NaN,NaN,(주)비츠로 대표이사,"마산여자상업고등학교졸업(현,마산무학여자고등학교)",(현) (주)비츠로대표이사,(현) 광명전기(주)감사,False
3,강감창,姜玪昌,19620305,남,52,시·도의회의원선거,20140604,Legislator,서울특별시,송파구,...,등록,1,14552.0,55.17,NaN,건축사,인하대학교 대학원 건축공학과 졸업(공학석사),(전)서울시의회 건설위원장,(현)서울시의원,https://namu.wiki/w/%EA%B0%95%EA%B0%90%EC%B0%BD
4,강갑중,姜甲中,19490322,남,65,구·시·군의회의원선거,20140604,Legislator,경상남도,진주시,...,등록,1,7238.0,28.88,NaN,정치인,"경상대학(현, 경상대학교)농화학과 졸업",(전)경남도의회 혁신도시 건설위원장,(전)한나라당 중앙위원회 농수산 해양위원장,https://namu.wiki/w/%EA%B0%95%EA%B0%91%EC%A4%91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10109,황환주,黃煥周,19570417,남,57,구·시·군의회의원선거,20140604,Legislator,강원도,춘천시,...,등록,1,2913.0,25.86,NaN,춘천시의회의원,성수고등학교졸업,(전)남산우체국장,(현)춘천시의회의원,NaN
10110,황효진,黃孝振,19800430,여,34,구·시·군의회의원선거,20140604,Legislator,경기도,안산시상록구,...,등록,0,NaN,NaN,NaN,안산시의회의원,서울대학교 대학원 사회복지학과 졸업(사회복지학석사),(현) 안산시의회 새누리당 대변인,(전) 6대 안산시의회 예산결산 특별위원장,NaN
10111,황흥구,黃興九,19520625,남,61,시·도의회의원선거,20140604,Legislator,인천광역시,남동구,...,등록,1,20947.0,50.07,NaN,수필가,인하대학교 행정대학원 졸업(행정학석사),(전)인천시 남동구 부구청장,(전)인천대학교 사무처장,NaN
10112,황흥섭,黃興燮,19491211,남,64,구·시·군의회의원선거,20140604,Legislator,서울특별시,용산구,...,등록,0,NaN,NaN,NaN,정당인,경북 풍기고등학교 졸업,(전)용산구 의회 4대 의원,(현)용산구 영남향우회 부회장,NaN


# test set for sk politicians

In [144]:
# test = sktest

In [145]:
# test

# scrape namuwiki with selenium

In [146]:
url_stem = "https://namu.wiki"

p_error_name = "data-v-1ac09f2e"
p_error_text = "해당 문서를 찾을 수 없습니다."

error_count = 0

In [ ]:
# instantiate selenium webdriver
service = Service(executable_path=ChromeDriverManager().install())

# scrape namuwiki using selenium
for index,p in sktest.iterrows():
    
    if(error_count>10):
        break
    
    if(pd.isnull(p.URL_NamuWiki)):
    
        # construct url
        url= url_stem+"/w/"+urllib.parse.quote(p.Name_Native.encode("utf-8"))

        # reset content & has_content_flag
        has_content_flag = True
        content=""
        
        try:

            # request url using selenium webdriver
            driver = webdriver.Chrome(service=service)
            driver.get(url)

            # check if page has content
            elements = driver.find_elements(By.TAG_NAME, 'p')

            for e in elements:
                if(e.text==p_error_text):
                    has_content_flag = False

            if(has_content_flag):
                content = driver.page_source

            # close webdriver
            driver.quit()

            # save content
            if(has_content_flag):

                # save output as an html file
                fout = open(fileout_path+"seleniumtest_"+str(index)+"_"+p.Name_Native+".html","w",encoding="utf-8")
                fout.write(content)
                fout.close()

                # update HasNamuWiki content field in dataframe
                sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=url

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Has Content")
                

            else:

                 # update HasNamuWiki content field in dataframe
                sktest.loc[sktest.Name_Native == p.Name_Native,'URL_NamuWiki']=False

                print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Empty")
                
            error_count = 0

        except:
                        
            print(index,p.Name_Native,url,"Number of <p> elements:",len(elements),"Error")
            
            error_count = error_count + 1

            



        #cookie = {l["name"]:l["value"] for l in driver.get_cookies()}
        #print(cookie)

    


61 강민국 https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD Number of <p> elements: 4 Error
62 강민수 https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EC%88%98 Number of <p> elements: 4 Has Content
63 강민아 https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EC%95%84 Number of <p> elements: 3 Has Content
64 강민영 https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EC%98%81 Number of <p> elements: 4 Has Content
65 강민영 https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EC%98%81 Number of <p> elements: 4 Has Content
66 강방원 https://namu.wiki/w/%EA%B0%95%EB%B0%A9%EC%9B%90 Number of <p> elements: 5 Empty
67 강범석 https://namu.wiki/w/%EA%B0%95%EB%B2%94%EC%84%9D Number of <p> elements: 4 Has Content
68 강병국 https://namu.wiki/w/%EA%B0%95%EB%B3%91%EA%B5%AD Number of <p> elements: 5 Empty
69 강병기 https://namu.wiki/w/%EA%B0%95%EB%B3%91%EA%B8%B0 Number of <p> elements: 4 Has Content
70 강병수 https://namu.wiki/w/%EA%B0%95%EB%B3%91%EC%88%98 Number of <p> elements: 4 Has Content
71 강병원 https://namu.wiki/w/%EA%B0%95%EB%B3%91%EC%9B%90 Number of <p> elements: 4 Has Co

152 강연종 https://namu.wiki/w/%EA%B0%95%EC%97%B0%EC%A2%85 Number of <p> elements: 5 Empty
153 강연호 https://namu.wiki/w/%EA%B0%95%EC%97%B0%ED%98%B8 Number of <p> elements: 5 Empty
154 강연희 https://namu.wiki/w/%EA%B0%95%EC%97%B0%ED%9D%AC Number of <p> elements: 4 Has Content
155 강영광 https://namu.wiki/w/%EA%B0%95%EC%98%81%EA%B4%91 Number of <p> elements: 5 Empty
156 강영근 https://namu.wiki/w/%EA%B0%95%EC%98%81%EA%B7%BC Number of <p> elements: 5 Empty
157 강영봉 https://namu.wiki/w/%EA%B0%95%EC%98%81%EB%B4%89 Number of <p> elements: 5 Empty
158 강영삼 https://namu.wiki/w/%EA%B0%95%EC%98%81%EC%82%BC Number of <p> elements: 5 Empty
159 강영석 https://namu.wiki/w/%EA%B0%95%EC%98%81%EC%84%9D Number of <p> elements: 4 Has Content
160 강영석 https://namu.wiki/w/%EA%B0%95%EC%98%81%EC%84%9D Number of <p> elements: 4 Has Content
161 강영수 https://namu.wiki/w/%EA%B0%95%EC%98%81%EC%88%98 Number of <p> elements: 4 Has Content
162 강영순 https://namu.wiki/w/%EA%B0%95%EC%98%81%EC%88%9C Number of <p> elements: 5 Empty
163 강영옥 

243 강한구 https://namu.wiki/w/%EA%B0%95%ED%95%9C%EA%B5%AC Number of <p> elements: 5 Empty
244 강한옥 https://namu.wiki/w/%EA%B0%95%ED%95%9C%EC%98%A5 Number of <p> elements: 5 Empty
245 강혁 https://namu.wiki/w/%EA%B0%95%ED%98%81 Number of <p> elements: 4 Has Content
246 강현분 https://namu.wiki/w/%EA%B0%95%ED%98%84%EB%B6%84 Number of <p> elements: 5 Empty
247 강현삼 https://namu.wiki/w/%EA%B0%95%ED%98%84%EC%82%BC Number of <p> elements: 5 Empty
248 강현석 https://namu.wiki/w/%EA%B0%95%ED%98%84%EC%84%9D Number of <p> elements: 4 Has Content
249 강현태 https://namu.wiki/w/%EA%B0%95%ED%98%84%ED%83%9C Number of <p> elements: 4 Has Content
250 강형식 https://namu.wiki/w/%EA%B0%95%ED%98%95%EC%8B%9D Number of <p> elements: 5 Empty
251 강형욱 https://namu.wiki/w/%EA%B0%95%ED%98%95%EC%9A%B1 Number of <p> elements: 4 Has Content
252 강혜란 https://namu.wiki/w/%EA%B0%95%ED%98%9C%EB%9E%80 Number of <p> elements: 5 Empty
253 강혜련 https://namu.wiki/w/%EA%B0%95%ED%98%9C%EB%A0%A8 Number of <p> elements: 5 Empty
254 강혜순 https://na

In [138]:
# sktest

In [139]:
sktest.to_excel(filein_path+"sk_testset.xlsx",index=False)

In [ ]:
# things to check

# 1. recorded links are valid (no false positives); non-recorded links are actually empty (no false negatives)


# 2. disambiguation? what about names that link to more than one person?
#       - disambiguation links are usually included in the page
#       - e.g. https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD - 강민국
#       - 정치인에 대한 내용은 강민국(정치인) 문서를 참고하십시오.
#       - https://namu.wiki/w/%EA%B0%95%EB%AF%BC%EA%B5%AD(%EC%A0%95%EC%B9%98%EC%9D%B8) - 강민국(정치인)